In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install langchain
!{sys.executable} -m pip install openai
!{sys.executable} -m pip install huggingface_hub
!{sys.executable} -m pip install google-api-python-client

In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''
os.environ['GOOGLE_API_KEY'] = ''

In [16]:
from langchain.llms import OpenAI
from langchain import HuggingFaceHub, LLMChain, PromptTemplate
import openai
import requests
from langchain.chat_models import ChatOpenAI

In [22]:
template = """Question: {question}
Please provide Answer as a sentence.
Answer: """
prompt = PromptTemplate(
        template=template,
    input_variables=['question']
)

def transform_llm_result(generations):
    answers = []
    for generation in generations:
        answer = generation[0].text
        answers.append(answer)
    return answers

def ask_llm(chain, qs):
    result = chain.generate(qs)
    generations = result.generations
    return transform_llm_result(generations)

def call_google_fact_checker(generations):
    for claim in generations:
        API_URL = 'https://factchecktools.googleapis.com/v1alpha1/claims:search'
        print(claim)

        params = {
            'key': os.environ.get('GOOGLE_API_KEY'),
            'query': claim,
            'languageCode': 'en-US'
        }

        response = requests.get(API_URL, params=params)
        if response.status_code == 200:
            print('Statement is true')
            body = response.json()
            print(body)
        else:
            print ('Statement is false')


In [23]:
# initialize Hub LLM
hub_llm = HuggingFaceHub(
   repo_id='google/flan-t5-xxl',
    model_kwargs={'temperature':1e-10}
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# initialize the models
davinci = OpenAI(
    temperature=0,
    model_name="text-davinci-003"
)

davinci_llm_chain = LLMChain(
    prompt=prompt,
    llm=davinci
)

### intialize gpt3 model

gpt = ChatOpenAI(
    temperature=0,
    model='gpt-3.5-turbo'
)

gpt_llm_chain = LLMChain(
    prompt=prompt,
    llm=gpt
)
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]

    
output = ask_llm(llm_chain, qs)
op = ask_llm(davinci_llm_chain, qs)
op1 = ask_llm(gpt_llm_chain, qs)

#compare the results
if op == output: 
    print ("Davinci and Flan have same answers") 
elif op == op1: 
    print ("Davinci and GPT have same answers") 
elif output == op1: 
    print ("Flan and GPT have same answers") 
else: 
    print ("Davinci, GPT and Flan have different answers") 
    
print("checking davinci answers")
call_google_fact_checker(op)
print("checking flan answers")
call_google_fact_checker(output)    
print("checking gpt answers")
call_google_fact_checker(op1)   

Davinci, GPT and Flan have different answers
checking davinci answers
 The Green Bay Packers won the Super Bowl in the 2010 season.
Statement is true
{}
 I am 193.04 centimeters tall.
Statement is true
{}
 The 12th person on the moon was Harrison Schmitt, an American geologist and astronaut.
Statement is true
{}
 A blade of grass does not have any eyes.
Statement is true
{}
checking flan answers
The New England Patriots won the Super Bowl in the 2010 season.
Statement is true
{}
If I am 6 ft 4 inches, I am 184 centimeters tall.
Statement is true
{}
Neil Armstrong was the 12th person to walk on the moon
Statement is true
{}
A blade of grass has two eyes.
Statement is true
{}
checking gpt answers
The Green Bay Packers won the Super Bowl in the 2010 season.
Statement is true
{}
You are approximately 193 centimeters tall.
Statement is true
{}
The 12th person on the moon was Eugene Cernan, who was part of the Apollo 17 mission in 1972.
Statement is true
{}
A blade of grass does not have any